# Naive word2vec

This task can be formulated very simply. Follow this [paper](https://arxiv.org/pdf/1411.2738.pdf) and implement word2vec like a two-layer neural network with matrices $W$ and $W'$. One matrix projects words to low-dimensional 'hidden' space and the other - back to high-dimensional vocabulary space.

![word2vec](https://i.stack.imgur.com/6eVXZ.jpg)

You can use TensorFlow/PyTorch (numpy too, if you love to calculate gradients on your own and want some extra points, but don't forget to numerically check your gradients) and code from your previous task. Again: you don't have to implement negative sampling (you may reduce your vocabulary size for faster computation).

**Results of this task**:
 * trained word vectors (mention somewhere, how long it took to train)
 * plotted loss (so we can see that it has converged)
 * function to map token to corresponding word vector
 * beautiful visualizations (PCE, T-SNE), you can use TensorBoard and play with your vectors in 3D (don't forget to add screenshots to the task)
 * qualitative evaluations of word vectors: nearest neighbors, word analogies

**Extra:**
 * quantitative evaluation:
   * for intrinsic evaluation you can find datasets [here](https://aclweb.org/aclwiki/Analogy_(State_of_the_art))
   * for extrincis evaluation you can use [these](https://medium.com/@dataturks/rare-text-classification-open-datasets-9d340c8c508e)

Also, you can find any other datasets for quantitative evaluation. If you chose to do this, please use the same datasets across tasks 3, 4, 5 and 6.

Again. It is **highly recommended** to read this [paper](https://arxiv.org/pdf/1411.2738.pdf)

Example of visualization in tensorboard:
https://projector.tensorflow.org

Example of 2D visualisation:

![2dword2vec](https://www.tensorflow.org/images/tsne.png)

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [ ]:
from skipgram import SkipGram, SkipGramBatcher
import torch
import gc
import datetime
import pickle

In [ ]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

#### Constants

In [ ]:
VOCAB_SIZE = 5000
BATCH_SIZE = 50
EMBEDDINGS_DIM = 100
EPOCH_NUM = 2
WINDOW_SIZE = 2
LOGS_PERIOD = 500

#### Load corpus into batcher

In [ ]:
text = []
with open('./data/text8', 'r') as text8:
    text = text8.read().split()

# text = ['first', 'used', 'against', 'early', 'working', 'radicals', 'including', 'class', 'other']
batcher = SkipGramBatcher(corpus=text, vocab_size=VOCAB_SIZE, batch_size=BATCH_SIZE, window_size=WINDOW_SIZE)

# free memory
text = []
gc.collect()

#### Create and train model

In [ ]:
model

In [ ]:
loss_history = []
corpus_size = len(batcher.corpus_indexes)

model = SkipGram(VOCAB_SIZE, EMBEDDINGS_DIM)
loss_fun = torch.nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
learning_started = datetime.datetime.now()

cumulative_loss = 0
for epoch in range(EPOCH_NUM):
    for i, (context, target) in enumerate(batcher):
        tensor_context = torch.from_numpy(context).type(torch.LongTensor)
        tensor_target = torch.from_numpy(target).type(torch.LongTensor)

        model.zero_grad()

        log_probs = model(tensor_context)
        loss = loss_fun(log_probs, tensor_target)
        loss.backward()
        optimizer.step()
        cumulative_loss += loss

        if i % LOGS_PERIOD == 0:
            print(f'Cumulative loss on {(i * BATCH_SIZE / corpus_size) * 100:.1f}%: {cumulative_loss:.3f}')
            loss_history.append(loss.data)
            cumulative_loss = 0

In [ ]:
learning_ended = datetime.datetime.now()
learning_time = (learning_ended - learning_started).total_seconds()
learning_ended = learning_ended.strftime("%H-%M %d-%m-%Y")

#### Saving model

In [ ]:
torch.save(model, f'./models/skipgram(epoch_num-{EPOCH_NUM})(vocab-{VOCAB_SIZE})(batch-{BATCH_SIZE})' + \
           f'(emb-{EMBEDDINGS_DIM})(wind-{WINDOW_SIZE})(consumed-{learning_time})'+ \
           f'(finished-{learning_ended}).pytorchmodel')

In [ ]:
with open(f'./data/loss/loss_history(epoch_num-{EPOCH_NUM})(vocab-{VOCAB_SIZE})(batch-{BATCH_SIZE})' + \
           f'(emb-{EMBEDDINGS_DIM})(wind-{WINDOW_SIZE})(consumed-{learning_time})'+ \
           f'(finished-{learning_ended}).pickle', 'wb') as f:
    pickle.dump(loss_history, f)